In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import numpy as np
import pandas as pd
import re

In [2]:
DRIVER_PATH=r'C:\Users\KacperSokolowski\Desktop\&&&\chrome_driver/chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)

site = 'https://www.otodom.pl/pl/oferty/wynajem/mieszkanie/warszawa?distanceRadius=0&page=1&limit=36&market=ALL&locations=%5Bcities_6-26%5D&viewType=listing'
driver.get(site)
sleep(1)

# Akceptacja plików cookie na stronie
driver.find_element(By.XPATH, '//button[text()="Akceptuję"]').click()
sleep(1)

# wyszukiwanie linków do ogłoszeń na 1 stronie
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
sleep(1)
ts=driver.find_elements(By.XPATH, '//a[@data-cy="listing-item-link"]')

announcements_links = []
for t in ts:
    announcements_links.append(t.get_attribute('href'))

# pętla na przechodzenie do dalczych stron i zbieranie tam linków - duży sleep, bo java script na stornie nie chciał się ławdować i scraper wariował
for i in range(2,100):
    try:
        driver.get(f'https://www.otodom.pl/pl/oferty/wynajem/mieszkanie/warszawa?distanceRadius=0&page={i}&limit=36&market=ALL&locations=%5Bcities_6-26%5D&viewType=listing')
        sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(1)
        ts=driver.find_elements(By.XPATH, '//a[@data-cy="listing-item-link"]')
        for t in ts:
            announcements_links.append(t.get_attribute('href'))
    except:
        pass

# usuwanie duplikatów linków do ogłoszeń
announcements_links = list(dict.fromkeys(announcements_links))

# funkcja do wcyiągania cyfr i zmiany separatora decymalnego
def get_numbers(string):
    extract = ""
    for s in string:
        if s.isdigit() or s == ",":
            extract = extract + s
    return(extract.replace(',', '.', 1))

# przygotowywanie list dla wartości z ogłoszeń
cena_wynajmu = []
powierzchnia = []
lokalizacja = []
lokalizacja2 = []
liczba_pokoi = []
pietro = []
wysokosc_budynku = []
rodzaj_zabudowy = []
wyposazenie = []
rok_budowy = []
czy_winda = []
herf = []
latitude = []
longitude = []

# pętla, która zbiera wartości z ogłoszeń
#3
for i in range(0,len(announcements_links)):
    try:
        new_site = announcements_links[i]
        driver.get(new_site)
        sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(1)
        loc_map = driver.find_element(By.ID, 'map')
        loc_map.location_once_scrolled_into_view
        sleep(1)
        try:
            location_container = driver.find_element(By.XPATH, '//a[@class="e1nbpvi60 css-171pgf6 e1enecw71"]').text
            rent_container = driver.find_element(By.XPATH, '//strong[@aria-label="Cena"]').text
            area_container = driver.find_element(By.XPATH, '//div[@aria-label="Powierzchnia"]').text
            room_num_container = driver.find_element(By.XPATH, '//div[@aria-label="Liczba pokoi"]').text
            floor_container = driver.find_element(By.XPATH, '//div[@aria-label="Piętro"]').text
            building_type_container = driver.find_element(By.XPATH, '//div[@aria-label="Rodzaj zabudowy"]').text
            furnishings_container = driver.find_element(By.XPATH, '//div[@aria-label="Wyposażenie"]').text
            year_of_construction_container = driver.find_element(By.XPATH, '//div[@aria-label="Rok budowy"]').text
            elevator_container = driver.find_element(By.XPATH, '//div[@aria-label="Winda"]').text
            
            map_link_elem = driver.find_elements(By.XPATH, '//a[@title="Pokaż ten obszar w Mapach Google (otwiera się w nowym oknie)"]')
            map_link = [elem.get_attribute('href') for elem in map_link_elem]
            latitude_container = re.search('ll=(.*),',str(map_link[0])).group(1)
            longitude_container = re.search(',(.*)&z',str(map_link[0])).group(1)

            herf.append(announcements_links[i])
            cena_wynajmu.append(get_numbers(rent_container))
            powierzchnia.append(get_numbers(area_container)[:-1])
            lokalizacja.append(location_container.split(", ")[1])
            lokalizacja2.append(location_container.split(", ")[0])
            liczba_pokoi.append(get_numbers(room_num_container))
            
            latitude.append(latitude_container)
            longitude.append(longitude_container)
            
            rodzaj_zabudowy.append(building_type_container.replace('''Rodzaj zabudowy
''', '', 1))
            wyposazenie.append(furnishings_container.replace('''Wyposażenie
''', '', 1))
            rok_budowy.append(get_numbers(year_of_construction_container))
            czy_winda.append(elevator_container.replace('''Winda
''', '', 1))
            if "/" in floor_container:
                wysokosc_budynku.append(floor_container.split("/")[1])
                if 'parter' in floor_container.split("/")[0]:
                    pietro.append('0')
                else:
                    pietro.append(get_numbers(floor_container.split("/")[0]))
            else:
                pietro.append(get_numbers(floor_container))
                wysokosc_budynku.append(get_numbers(floor_container))
        except:
            pass
    except:
        pass

# tworzenie dataframe'u z list

df = pd.DataFrame({
    "link": herf,
    "cena_wynajmu": cena_wynajmu,
    "powierzchnia": powierzchnia,
    "lokalizacja": lokalizacja,
    "lokalizacja2": lokalizacja2,
    "liczba_pokoi": liczba_pokoi,
    "rodzaj_zabudowy": rodzaj_zabudowy,
    "wyposazenie": wyposazenie,
    "rok_budowy": rok_budowy,
    "czy_winda": czy_winda,
    "wysokosc_budynku": wysokosc_budynku,
    "pietro": pietro,
    "latitude":latitude,
    "longitude":longitude
})

# Dalej; uzupełnienie datafram'u o brakujace informacje, których nie dało się wyciągnąć z samych ogłoszeń

####### klima

ogloszenia_z_klimatyacja = []

# pętla na dalesze strony
for i in range(1,30):
    try:
        driver.get(f'https://www.otodom.pl/pl/oferty/wynajem/mieszkanie/warszawa?distanceRadius=0&page={i}&limit=36&market=ALL&locations=%5Bcities_6-26%5D&media=%5B%5D&extras=%5BAIR_CONDITIONING%5D&viewType=listing')
        sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(1)
        ts=driver.find_elements(By.XPATH, '//a[@data-cy="listing-item-link"]')
        for t in range(3,39):
            try:
                ogloszenia_z_klimatyacja.append(ts[t].get_attribute('href'))
            except IndexError:
                break
    except:
        pass

# usuwanie duplikatów
ogloszenia_z_klimatyacja = list(dict.fromkeys(ogloszenia_z_klimatyacja))

#dodanie nowej kolumny do df, kóra przyjmuje wartość True jeśli ogłoszenie z klimatyzacją dopasowuje się do linku ogłoszenia w dfreimie
df['czy_klimatyzacja'] = df['link'].isin(ogloszenia_z_klimatyacja)

#zmiana wartości z boolean na int
df['czy_klimatyzacja'] = df['czy_klimatyzacja'].astype(int)

#zapisanie df'u jako plik csv
df.to_csv('C:/Users/kacpe/Desktop/Projects/R_project/test.csv',
          na_rep='NaN',
          index=False,
          encoding='utf-8'
)

<ipython-input-2-f41574e4cc84>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH)


In [3]:
df

,link,cena_wynajmu,powierzchnia,lokalizacja,lokalizacja2,liczba_pokoi,rodzaj_zabudowy,wyposazenie,rok_budowy,czy_winda,wysokosc_budynku,pietro,latitude,longitude,czy_klimatyzacja
0,https://www.otodom.pl/pl/oferta/wynajme-kawalk...,2400,21,Warszawa,Praga-Północ,1,kamienica,"lodówka, meble, piekarnik, kuchenka, pralka",,nie,4,3,52.2633,21.02832,0
1,https://www.otodom.pl/pl/oferta/mieszkanie-urs...,3200,46,Warszawa,Ursus,2,blok,"lodówka, meble, piekarnik, kuchenka, telewizor...",2001,nie,7,7,52.19512,20.88379,1
2,https://www.otodom.pl/pl/oferta/2-pokojowe-mie...,2700,53,Praga-Południe,Warszawa,2,Zapytaj,"zmywarka, lodówka, meble, piekarnik, kuchenka,...",,nie,5,4,52.237892,21.104632,0
3,https://www.otodom.pl/pl/oferta/mieszkanie-1-p...,2300,25,Warszawa,Słodowiec,1,apartamentowiec,"zmywarka, lodówka, meble, pralka",2022,tak,5,2,52.27414,20.96259,0
4,https://www.otodom.pl/pl/oferta/ladne-nice-and...,5100,36,Śródmieście,Warszawa,2,apartamentowiec,meble,2018,tak,12,3,52.236062,20.997513,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3446,https://www.otodom.pl/pl/oferta/kawalerka-ID4jRW0,3200,35,Warszawa,Mokotów,1,kamienica,"lodówka, meble, piekarnik, kuchenka, pralka",,nie,3,1,52.19832,21.01715,0
3447,https://www.otodom.pl/pl/oferta/wynajme-od-zar...,6500,105,Warszawa,Pole Mokotowskie,5,kamienica,"zmywarka, lodówka, meble, piekarnik, kuchenka,...",1932,tak,6,1,52.2119,21.00553,0
3448,https://www.otodom.pl/pl/oferta/apartament-z-d...,5900,76,Żoliborz,Warszawa,3,apartamentowiec,"zmywarka, lodówka, meble, piekarnik, kuchenka,...",2012,tak,5,4,52.26971,20.98462,1
3449,https://www.otodom.pl/pl/oferta/wynajme-mieszk...,4000,90,Ursynów,Warszawa,3,blok,"zmywarka, lodówka, meble, piekarnik, kuchenka",2004,tak,3,0,52.145626,21.063004,0


In [75]:
df = df.dropna().reset_index(drop=True)

df['location'] = [a if a!='Warszawa' else b for a, b in zip(df['lokalizacja'],df['lokalizacja2'])]

# df['czy_zmywarka'] = df['wyposazenie'].str.contains('zmywarka')
# df['czy_tv']       = df['wyposazenie'].str.contains('telewizor')
# df['czy_winda']    = df['czy_winda'].str.contains('tak')

int_cols   = ['cena_wynajmu','liczba_pokoi','rok_budowy','czy_winda','wysokosc_budynku','pietro','czy_klimatyzacja','czy_zmywarka','czy_tv']
float_cols = ['powierzchnia', 'latitude','longitude']

df[int_cols]   = df[int_cols].astype(int)
df[float_cols] = df[float_cols].astype(float)

rename_dict = {'cena_wynajmu'     : 'rent_price',
               'powierzchnia'     : 'area',
               'liczba_pokoi'     : 'rooms_num',
               'rodzaj_zabudowy'  : 'building_type',
               'rok_budowy'       : 'construction_year',
               'czy_winda'        : 'elevator',
               'wysokosc_budynku' : 'building_height',
               'pietro'           : 'floor',
               'czy_klimatyzacja' : 'air_conditioning',
               'czy_zmywarka'     : 'dishwasher',
               'czy_tv'           : 'tv'
               }

df.rename(columns=rename_dict, inplace=True)

df['premium'] = np.random.randint(2, size = df.shape[0])
df.insert(0, 'id', df.index+1)

df

,id,link,rent_price,area,lokalizacja,lokalizacja2,rooms_num,building_type,wyposazenie,construction_year,elevator,building_height,floor,latitude,longitude,air_conditioning,dishwasher,tv,location,premium
0,1,https://www.otodom.pl/pl/oferta/mieszkanie-urs...,3200,46,Warszawa,Ursus,2,blok,"lodówka, meble, piekarnik, kuchenka, telewizor...",2001,0,7,7,52.195120,20.883790,1,0,1,Ursus,0
1,2,https://www.otodom.pl/pl/oferta/mieszkanie-1-p...,2300,25,Warszawa,Słodowiec,1,apartamentowiec,"zmywarka, lodówka, meble, pralka",2022,1,5,2,52.274140,20.962590,0,1,0,Słodowiec,1
2,3,https://www.otodom.pl/pl/oferta/ladne-nice-and...,5100,36,Śródmieście,Warszawa,2,apartamentowiec,meble,2018,1,12,3,52.236062,20.997513,0,0,0,Śródmieście,1
3,4,https://www.otodom.pl/pl/oferta/bardzo-interes...,7000,38,Śródmieście,Warszawa,2,apartamentowiec,meble,2009,1,7,6,52.236062,20.997513,0,0,0,Śródmieście,0
4,5,https://www.otodom.pl/pl/oferta/dobra-cena-2-s...,5000,110,Żoliborz,Warszawa,4,blok,brak informacji,1985,1,10,10,52.270983,20.967038,0,0,0,Żoliborz,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2586,2587,https://www.otodom.pl/pl/oferta/warszawa-okrze...,2600,41,Praga-Północ,Warszawa,1,apartamentowiec,"zmywarka, lodówka, meble, piekarnik, kuchenka,...",2009,1,4,1,52.250421,21.033205,0,1,1,Praga-Północ,1
2587,2588,https://www.otodom.pl/pl/oferta/wynajme-od-zar...,6500,105,Warszawa,Pole Mokotowskie,5,kamienica,"zmywarka, lodówka, meble, piekarnik, kuchenka,...",1932,1,6,1,52.211900,21.005530,0,1,1,Pole Mokotowskie,1
2588,2589,https://www.otodom.pl/pl/oferta/apartament-z-d...,5900,76,Żoliborz,Warszawa,3,apartamentowiec,"zmywarka, lodówka, meble, piekarnik, kuchenka,...",2012,1,5,4,52.269710,20.984620,1,1,1,Żoliborz,1
2589,2590,https://www.otodom.pl/pl/oferta/wynajme-mieszk...,4000,90,Ursynów,Warszawa,3,blok,"zmywarka, lodówka, meble, piekarnik, kuchenka",2004,1,3,0,52.145626,21.063004,0,1,0,Ursynów,0


In [94]:
adv_cols = ['id','link','premium']
loc_cols = ['id','latitude','longitude']
par_cols = df.columns.drop(list(set(adv_cols).symmetric_difference(set(loc_cols)))+['lokalizacja','lokalizacja2','wyposazenie','building_height','construction_year'])

advertisements = df[adv_cols] 
location       = df[loc_cols]
parameters     = df[par_cols]

In [33]:
user_full = pd.DataFrame({
    "id"       : [1,2,3,4,5],
    "login"    : ['ks419067','ks419068','ks419069','ks419060','ks419061'],
    "password" : ['test1','test1','test1','test1','test1'],
    "premium"  : [1,0,1,0,1]
})

users   = user_full[['id','login','password']]
premium = user_full[['id','premium']]

In [21]:
import sqlite3
conn = sqlite3.connect('pythonproject.db')
cursor = conn.cursor()

In [128]:
cursor.execute('''
    CREATE TABLE advertisements (
        id INTEGER NOT NULL PRIMARY KEY,
        link VARCHAR(200),
        premium INTEGER);
''')

advertisements.to_sql('advertisements', conn, if_exists='append', index=False)

2591

In [131]:
cursor.execute('''
    CREATE TABLE location (
        id INTEGER NOT NULL PRIMARY KEY,
        latitude REAL,
        longitude REAL);
''')

location.to_sql('location', conn, if_exists='append', index=False)

2591

In [145]:
cursor.execute('''
    CREATE TABLE parameters (
        id INTEGER NOT NULL PRIMARY KEY,
        rent_price INTEGER,
        area INTEGER,
        rooms_num INTEGER,
        floor INTEGER,
        location VARCHAR(20),
        building_type VARCHAR(15),
        elevator BOOLEAN,
        air_conditioning BOOLEAN,
        dishwasher BOOLEAN,
        tv BOOLEAN);
''')

parameters.to_sql('parameters', conn, if_exists='append', index=False)

2591

In [ ]:
### I messed up a little bit - column building_type contains values in polish language
### Let's fix that

In [5]:
def extract_data(query : str) -> pd.DataFrame:
        conn = sqlite3.connect('pythonproject.db')
        df = pd.read_sql_query(query, conn)
        conn.close()
        return df

In [9]:
q = '''
    SELECT DISTINCT(building_type)
    FROM parameters
'''
df = extract_data(q)
df

,building_type
0,blok
1,apartamentowiec
2,kamienica
3,dom wolnostojący
4,None
5,szeregowiec
6,plomba
7,loft


In [31]:
update_dict = {
    'block'          : 'blok',
    'apartment'      : 'apartamentowiec',
    'tenement'       : 'kamienica',
    'detached house' : 'dom wolnostojący',
    'terraced house' : 'szeregowiec',
    'infill'         : 'plomba'
}

for k, v in update_dict.items():
    old = v
    new = k
    update_query = f'''
    UPDATE parameters
    SET building_type = '{new}'
    WHERE building_type = '{old}';
    '''
    cursor.execute(update_query)
    conn.commit()

In [32]:
q = '''
    SELECT DISTINCT(building_type)
    FROM parameters
'''
df = extract_data(q)
df

,building_type
0,block
1,apartment
2,tenement
3,detached house
4,None
5,terraced house
6,infill
7,loft


In [ ]:
### fixed

In [35]:
cursor.execute('''
    CREATE TABLE users (
        id INTEGER NOT NULL PRIMARY KEY,
        login VARCHAR(50) NOT NULL UNIQUE,
        password VARCHAR(50) NOT NULL);
''')

users.to_sql('users', conn, if_exists='append', index=False)

5

In [139]:
cursor.execute('''
    CREATE TABLE premium (
        id INTEGER NOT NULL PRIMARY KEY,
        premium INTEGER);
''')

premium.to_sql('premium', conn, if_exists='append', index=False)

5

In [ ]:
### Creating Views

In [38]:
flats_non_premium = '''
    CREATE VIEW flats_non_premium AS
    SELECT location.id, location.latitude, location.longitude, parameters.rent_price, parameters.area, parameters.rooms_num,
           parameters.floor, parameters.location, parameters.building_type, parameters.elevator, parameters.air_conditioning,
           parameters.dishwasher, parameters.tv, advertisements.link, advertisements.premium
    FROM location
    INNER JOIN parameters
    ON location.id = parameters.id
    INNER JOIN advertisements
    ON location.id = advertisements.id
    WHERE advertisements.premium = 0
'''
cursor.execute(flats_non_premium)

In [40]:
flats_premium = '''
    CREATE VIEW flats_premium AS
    SELECT location.id, location.latitude, location.longitude, parameters.rent_price, parameters.area, parameters.rooms_num,
           parameters.floor, parameters.location, parameters.building_type, parameters.elevator, parameters.air_conditioning,
           parameters.dishwasher, parameters.tv, advertisements.link, advertisements.premium
    FROM location
    INNER JOIN parameters
    ON location.id = parameters.id
    INNER JOIN advertisements
    ON location.id = advertisements.id
'''
cursor.execute(flats_premium)

In [43]:
users_full = '''
    CREATE VIEW users_full AS
    SELECT users.id, users.login, users.password, premium.premium
    FROM users
    INNER JOIN premium
    ON users.id = premium.id
'''
cursor.execute(users_full)